In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow as tf
import missingno as msno
from tensorflow import keras
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import  train_test_split

In [36]:
na_cat_value_dicts = {
    'Car_Owned':2,
    'Bike_Owned':2,
    'Active_Loan':2,
    'House_Own' : 2, # 더미값 대체
    'Accompany_Client':'##', # 더미값 으로 대체
    'Client_Income_Type':'NA',
    'Client_Education' : 'NA',
    'Client_Marital_Status' : 'NA',
    'Client_Gender' : 'XNA',
    'Loan_Contract_Type': 'NA',
    'Client_Housing_Type' : 'NA',
    'Homephone_Tag' : 2,
    'Workphone_Working' : 2,
    'Client_Occupation' : 'NA',
    'Cleint_City_Rating' : 2,  # 최빈값
    'Application_Process_Day' : 2, # 계약 요일 최빈값 
    'Application_Process_Hour' : 10 , # 계약 시간
    'Client_Permanent_Match_Tag' : 'NA', 
    'Client_Contact_Work_Tag' : 'NA',
    'Type_Organization' : 'XNA'
}

In [4]:
numeri_datas_cols_dicts = {
    'Client_Income' : 1.440000e+04,
    'Credit_Amount' :51750.000000, # 중앙값
    'Loan_Annuity' : 2499.750000, # 중앙값
    'Child_Count' : 0 ,# '최빈값,
    'Population_Region_Relative': 0.018850, # 중앙값
    'Age_Days' : 15733, #중앙값
    'Employed_Days' : 2211, #중앙값
    'Registration_Days' :  4492, #중앙값
    'ID_Days' : 3242.000000, # 중앙값
    'Own_House_Age' : -1, # 더미값
    'Client_Family_Members':1, # 최빈값
    'Score_Source_1' : -1 , #더미값
    'Score_Source_2' : 0.564978, # 중앙값
    'Score_Source_3' : 0.537070, # 중앙값
    'Social_Circle_Default' : 0, # 대출갚지못한 수 0으로 처리
    'Phone_Change': 755.000000, #중앙값
    'Credit_Bureau' : 0 # 0
}

In [5]:
data = pd.read_csv('after_pre_final.csv')

In [31]:
list(numeri_datas_cols_dicts.keys())

['Client_Income',
 'Credit_Amount',
 'Loan_Annuity',
 'Child_Count',
 'Population_Region_Relative',
 'Age_Days',
 'Employed_Days',
 'Registration_Days',
 'ID_Days',
 'Own_House_Age',
 'Client_Family_Members',
 'Score_Source_1',
 'Score_Source_2',
 'Score_Source_3',
 'Social_Circle_Default',
 'Phone_Change',
 'Credit_Bureau']

In [37]:
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
onehot_encoder = OneHotEncoder()
prep_processor = ColumnTransformer(
     [('standard',standard_scaler,list(numeri_datas_cols_dicts.keys())),
     ('onehot',onehot_encoder,list(na_cat_value_dicts.keys()))
     ]
)


In [33]:
X = data.drop('Default', axis=1)

In [12]:
Y = data['Default']

In [16]:
x_train, x_, y_train, y_ = train_test_split(X,Y,test_size=0.1, random_state = 5500, stratify=Y)

In [20]:
x_cv, x_test, y_cv, y_test=train_test_split(x_, y_,test_size=0.5, random_state = 5500, stratify=y_)

In [38]:
prep_processor.fit(x_train)

ColumnTransformer(transformers=[('standard', StandardScaler(),
                                 ['Client_Income', 'Credit_Amount',
                                  'Loan_Annuity', 'Child_Count',
                                  'Population_Region_Relative', 'Age_Days',
                                  'Employed_Days', 'Registration_Days',
                                  'ID_Days', 'Own_House_Age',
                                  'Client_Family_Members', 'Score_Source_1',
                                  'Score_Source_2', 'Score_Source_3',
                                  'Social_Circle_Default', 'Phone_Change',
                                  'Credit_Bur...
                                  'House_Own', 'Accompany_Client',
                                  'Client_Income_Type', 'Client_Education',
                                  'Client_Marital_Status', 'Client_Gender',
                                  'Loan_Contract_Type', 'Client_Housing_Type',
                                  'Homephone_Tag', 'Workphone_Working',
                                  'Client_Occupation', 'Cleint_City_Rating',
                                  'Application_Process_Day',
                                  'Application_Process_Hour',
                                  'Client_Permanent_Match_Tag',
                                  'Client_Contact_Work_Tag',
                                  'Type_Organization'])])

In [40]:
x_train_trans = prep_processor.transform(x_train)
x_cv_trans = prep_processor.transform(x_cv)
x_test_trans= prep_processor.transform(x_test)

In [46]:
x_train_trans.shape

(109670, 188)

In [110]:
model1 = tf.keras.models.Sequential(
    [
    keras.layers.Dense(units=188, input_shape=(188,), activation='relu', kernel_regularizer=keras.regularizers.L2(0)),
    keras.layers.Dense(units=376,  activation='relu', kernel_regularizer= keras.regularizers.L2(0)),
    keras.layers.Dense(units=500,  activation='relu', kernel_regularizer=keras.regularizers.L2(0) ),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units=1,activation='sigmoid'),
    ]
    , name='nbfi_predict'
)

In [127]:
model1.compile(optimizer="Adam", loss=keras.losses.binary_crossentropy,
metrics=[tf.keras.metrics.Accuracy(name='acc'),tf.keras.metrics.Precision(name='precision')
                          ,tf.keras.metrics.Recall(name='recall')] )


In [128]:
import os

In [129]:
checkpoint_path = "training_4/model_cp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1,)

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="acc", min_delta=0, patience=30)

In [130]:
y_train.to_numpy()

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [131]:
history = model1.fit(x_train_trans, y_train.to_numpy(),
                 validation_data=(x_cv_trans, y_cv.to_numpy()),epochs=600, callbacks=[cp_callback,earlystop_callback])

Epoch 1/600


c:\Users\PaleBlue\semi_project\venv\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/nbfi_predict/dense_8/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/nbfi_predict/dense_8/embedding_lookup_sparse/Reshape:0", shape=(None, 188), dtype=float32), dense_shape=Tensor("gradient_tape/nbfi_predict/dense_8/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


3420/3428 [============================>.] - ETA: 0s - loss: 0.1899 - acc: 3.8377e-04 - precision: 0.7600 - recall: 0.2948
Epoch 1: saving model to training_4\model_cp.ckpt
INFO:tensorflow:Assets written to: training_4\model_cp.ckpt\assets
3428/3428 [==============================] - 18s 5ms/step - loss: 0.1900 - acc: 3.8297e-04 - precision: 0.7595 - recall: 0.2945 - val_loss: 0.3112 - val_acc: 4.9237e-04 - val_precision: 0.3117 - val_recall: 0.0974
Epoch 2/600
3420/3428 [============================>.] - ETA: 0s - loss: 0.1776 - acc: 6.8531e-04 - precision: 0.7858 - recall: 0.3415
Epoch 2: saving model to training_4\model_cp.ckpt
INFO:tensorflow:Assets written to: training_4\model_cp.ckpt\assets
3428/3428 [==============================] - 19s 5ms/step - loss: 0.1775 - acc: 6.8387e-04 - precision: 0.7857 - recall: 0.3414 - val_loss: 0.3359 - val_acc: 4.9237e-04 - val_precision: 0.3413 - val_recall: 0.1156
Epoch 3/600
3348/3428 [============================>.] - ETA: 0s - loss: 0.1656 

In [102]:
x_cv_trans

<6093x188 sparse matrix of type '<class 'numpy.float64'>'
	with 225441 stored elements in Compressed Sparse Row format>